# Signal Visualization Dashboard

Objective:
- Visualize how current signal setups behave across 30d, 90d, and 180d windows.
- Spot stable edges vs fading edges quickly.

This notebook reads:
- `data/signal_queue.csv` for recent signal list
- `data/backtest_noregime/backtest_<SYMBOL>_<SETUP>_<WINDOW>d.csv` for trade outcomes


In [ ]:
from __future__ import annotations

from pathlib import Path

import numpy as np
import pandas as pd

HAS_PLOT = True
try:
    import matplotlib.pyplot as plt
    plt.style.use("seaborn-v0_8-whitegrid")
except Exception as exc:
    HAS_PLOT = False
    plt = None
    print(f"matplotlib unavailable ({exc}); plot cells will be skipped.")

pd.set_option("display.max_rows", 200)
pd.set_option("display.width", 140)


def find_repo_root(start: Path) -> Path:
    candidates = [start, *start.parents]
    for path in candidates:
        if (path / "main.py").exists() and (path / "config.json").exists():
            return path
    return start


ROOT = find_repo_root(Path.cwd())
QUEUE_PATH = ROOT / "data" / "signal_queue.csv"
BT_DIR = ROOT / "data" / "backtest_noregime"
WINDOWS = [30, 90, 180]

print(f"repo root: {ROOT}")
print(f"queue: {QUEUE_PATH}")
print(f"backtests: {BT_DIR}")


In [ ]:
queue = pd.read_csv(QUEUE_PATH)
queue["created_ts"] = pd.to_datetime(queue["created_ts"], utc=True, errors="coerce")

signals = (
    queue.sort_values("created_ts")
    .drop_duplicates(subset=["symbol", "setup_name"], keep="last")
    [["signal_id", "created_ts", "symbol", "setup_name", "status", "qty", "decision_reason"]]
    .reset_index(drop=True)
)

signals


In [ ]:
def backtest_path(symbol: str, setup: str, window: int) -> Path:
    return BT_DIR / f"backtest_{symbol}_{setup}_{window}d.csv"


def summarize_backtest(path: Path) -> dict:
    if not path.exists():
        return {
            "trades": np.nan,
            "win_rate": np.nan,
            "avg_r": np.nan,
            "median_r": np.nan,
            "best_r": np.nan,
            "worst_r": np.nan,
        }

    df = pd.read_csv(path)
    if df.empty:
        return {
            "trades": 0,
            "win_rate": np.nan,
            "avg_r": np.nan,
            "median_r": np.nan,
            "best_r": np.nan,
            "worst_r": np.nan,
        }

    r = pd.to_numeric(df.get("r_multiple"), errors="coerce").dropna()
    if r.empty:
        return {
            "trades": len(df),
            "win_rate": np.nan,
            "avg_r": np.nan,
            "median_r": np.nan,
            "best_r": np.nan,
            "worst_r": np.nan,
        }

    return {
        "trades": int(len(r)),
        "win_rate": float((r > 0).mean()),
        "avg_r": float(r.mean()),
        "median_r": float(r.median()),
        "best_r": float(r.max()),
        "worst_r": float(r.min()),
    }


rows = []
for _, sig in signals.iterrows():
    for window in WINDOWS:
        path = backtest_path(sig["symbol"], sig["setup_name"], window)
        stats = summarize_backtest(path)
        rows.append(
            {
                "signal_id": sig["signal_id"],
                "symbol": sig["symbol"],
                "setup": sig["setup_name"],
                "status": sig["status"],
                "window": window,
                "path": str(path),
                **stats,
            }
        )

summary = pd.DataFrame(rows)
summary


## Metric meaning

- `avg_r`: average R-multiple across trades in the window. Positive means average expectancy is positive.
- `median_r`: middle R-multiple. Useful to see typical trade quality when outliers distort averages.
- `win_rate`: share of trades with `r_multiple > 0`.

In this system, `R` means return measured in units of initial stop risk, not `R-squared`.


In [ ]:
pivot_avg = summary.pivot_table(
    index=["symbol", "setup"],
    columns="window",
    values="avg_r",
    aggfunc="first",
).sort_index()

if HAS_PLOT:
    ax = pivot_avg.plot(kind="bar", figsize=(12, 5), width=0.8)
    ax.set_title("Avg R by Signal and Window")
    ax.set_xlabel("Signal")
    ax.set_ylabel("Avg R")
    ax.axhline(0.0, color="black", linewidth=1)
    ax.legend(title="Window (days)")
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib for charts: uv add matplotlib")

pivot_avg


In [ ]:
pivot_win = summary.pivot_table(
    index=["symbol", "setup"],
    columns="window",
    values="win_rate",
    aggfunc="first",
).sort_index()

if HAS_PLOT:
    fig, ax = plt.subplots(figsize=(12, 4.5))
    im = ax.imshow(pivot_win.fillna(0).values, aspect="auto", cmap="YlGn", vmin=0, vmax=1)
    ax.set_title("Win Rate Heatmap (30/90/180)")
    ax.set_xticks(range(len(pivot_win.columns)))
    ax.set_xticklabels([str(c) for c in pivot_win.columns])
    ax.set_yticks(range(len(pivot_win.index)))
    ax.set_yticklabels([f"{s} | {u}" for s, u in pivot_win.index])
    ax.set_xlabel("Window (days)")
    ax.set_ylabel("Signal")
    plt.colorbar(im, ax=ax, label="Win rate")
    plt.tight_layout()
    plt.show()
else:
    pivot_win


In [ ]:
def load_r_series(symbol: str, setup: str, window: int) -> pd.Series:
    path = backtest_path(symbol, setup, window)
    if not path.exists():
        return pd.Series(dtype=float)
    df = pd.read_csv(path)
    r = pd.to_numeric(df.get("r_multiple"), errors="coerce").dropna().reset_index(drop=True)
    return r

long_rows = []
for _, sig in signals.iterrows():
    for window in WINDOWS:
        r = load_r_series(sig["symbol"], sig["setup_name"], window)
        for i, value in enumerate(r, start=1):
            long_rows.append(
                {
                    "signal": f"{sig['symbol']} | {sig['setup_name']}",
                    "window": window,
                    "trade_num": i,
                    "r_multiple": float(value),
                    "cum_r": float(r.iloc[:i].sum()),
                }
            )

trade_curves = pd.DataFrame(long_rows)

if HAS_PLOT and not trade_curves.empty:
    fig, ax = plt.subplots(figsize=(12, 5))
    for signal, g in trade_curves[trade_curves["window"] == 180].groupby("signal"):
        ax.plot(g["trade_num"], g["cum_r"], marker="o", linewidth=1.5, label=signal)
    ax.set_title("180d Cumulative R by Signal")
    ax.set_xlabel("Trade number")
    ax.set_ylabel("Cumulative R")
    ax.axhline(0.0, color="black", linewidth=1)
    ax.legend(loc="best", fontsize=8)
    plt.tight_layout()
    plt.show()

trade_curves.head()


In [ ]:
# Simple decision helper for quick triage in notebook

def classify(row30, row90, row180):
    if pd.isna(row30["avg_r"]) or pd.isna(row180["avg_r"]):
        return "missing-data"

    approve = (
        row90["trades"] >= 8
        and row180["trades"] >= 20
        and row90["avg_r"] >= 0.10
        and row180["avg_r"] >= 0.10
        and row180["median_r"] >= 0.0
    )
    hot_only = row30["trades"] >= 5 and row30["avg_r"] >= 0.30

    if approve:
        return "approve"
    if hot_only:
        return "hot-only"
    return "reject"

rows = []
for (symbol, setup), g in summary.groupby(["symbol", "setup"]):
    by_window = {int(r["window"]): r for _, r in g.iterrows()}
    row30 = by_window.get(30, pd.Series(dtype=float))
    row90 = by_window.get(90, pd.Series(dtype=float))
    row180 = by_window.get(180, pd.Series(dtype=float))
    rec = classify(row30, row90, row180)
    rows.append(
        {
            "symbol": symbol,
            "setup": setup,
            "status": g["status"].iloc[0],
            "rec": rec,
            "avg_r_30": row30.get("avg_r", np.nan),
            "avg_r_90": row90.get("avg_r", np.nan),
            "avg_r_180": row180.get("avg_r", np.nan),
            "median_r_180": row180.get("median_r", np.nan),
        }
    )

recommendations = pd.DataFrame(rows).sort_values(["rec", "symbol", "setup"])
recommendations


## Next actions

- Re-run this notebook after each daily scan or after new approvals.
- If a setup flips from `approve` to `hot-only` or `reject`, review recency and reduce/stop allocation.
- If 180d files are missing for a new signal, run backtests first so comparisons stay fair.
